In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torch.autograd import Variable
from data import VOCroot
from data import VOC_CLASSES as labelmap
import torch.utils.data as data
from PIL import Image
import sys
import os
from data import AnnotationTransform, VOCDetection, base_transform
from timeit import default_timer as timer
import argparse
import numpy as np
from ssd import build_ssd
import pickle
from collections import Counter, defaultdict

In [2]:
from utils.box_utils import jaccard, point_form

In [3]:
def voc_ap(rec, prec):
    """VOC_AP average precision calculations using 11-recall-point based AP
    metric (VOC2007)
    [precision integrated to recall]
    Params:
        rec (FloatTensor): recall cumsum
        prec (FloatTensor): precision cumsum
    Return:
        average precision (float)
    """
    ap = 0.
    for threshold in torch.range(0., 1., 0.1):
        if torch.sum(rec >= threshold) == 0:  # if no recs are >= this thresh
            p = 0
        else:
            # largest prec where rec >= thresh
            p = torch.max(prec[rec >= threshold])
        ap += p / 11.
    return ap

In [15]:
def test_net(net, cuda, valset, transform, top_k):
    # dump predictions and assoc. ground truth to text file for now
    num_images = len(valset)
    ovthresh = 0.5
    num_classes = 0

    # per class
    fp = defaultdict(list)
    tp = defaultdict(list)
    gts = defaultdict(list)
    precision = Counter()
    recall = Counter()
    ap = Counter()

    for i in range(5):
        confidence_threshold = 0.01
        print('Evaluating image {:d}/{:d}....'.format(i + 1, num_images))
        img = valset.pull_image(i)
        anno = valset.pull_anno(i)
        # print(anno)
        anno = torch.Tensor(anno).long()
        gt_classes = list(set(anno[:, 4]))
        x = Variable(transform(img).unsqueeze_(0))
        if cuda:
            x = x.cuda()
        y = net(x)  # forward pass
        detections = y.data
        # scale each detection back up to the image
        scale = torch.Tensor([img.size[0], img.size[1],
                              img.size[0], img.size[1]])
        # for each class
        if num_classes == 0:
            num_classes = detections.size(1)
        for cl in range(detections.size(1)):
            dets = detections[0, cl, :, :]
            mask = dets[:, 0].ge(0.01).expand(5, dets.size(0)).t()
            # all dets w > 0.01 conf for class
            dets = torch.masked_select(dets, mask).view(-1, 5)
            mask = anno[:, 4].eq(cl).expand(5, anno.size(0)).t()
            # all gts for class
            truths = torch.masked_select(anno, mask).view(-1, 5)
            if truths.numel() > 0:
                truths = truths[:, :-1]
                # gts[cl].extend([1] * truths.size(0))  # count gts
                if dets.numel() < 1:
                    continue  # no detections to count
                # there exist gt of this class in the image
                # check for tp & fp
                preds = dets[:, 1:]
                preds *= scale.unsqueeze(0).expand_as(preds)
                # compute overlaps
                overlaps = jaccard(truths.float(), preds)
                # if each gt obj is found yet
                found = [False] * overlaps.size(0)
                maxes = overlaps.max(0)
                for pb in range(overlaps.size(1)):
                    max_overlap = maxes[0][0, pb]
                    gt = maxes[1][0, pb]
                    if max_overlap > ovthresh:  # 0.5
                        if found[gt]:
                            # duplicate
                            fp[cl].append(1)
                            tp[cl].append(0)
                            gts[cl].append(0) # tp
                        else:
                            # not yet found
                            tp[cl].append(1)
                            fp[cl].append(0)
                            found[gt] = True  # mark gt as found
                            gts[cl].append(1) # tp
                    else:
                        fp[cl].append(1)
                        tp[cl].append(0)
                        gts[cl].append(0) # tp
            else:
                # there are no gts of this class in the image
                # all dets > 0.01 are fp
                if dets.numel() > 0:
                    fp[cl].extend([1] * dets.size(0))
                    tp[cl].extend([0] * dets.size(0))
                    gts[cl].extend([0] * dets.size(0))  # fn
    for cl in range(num_classes):
        if len(gts[cl]) < 1:
            continue
        # for each class calc rec, prec, ap
        tp_cumsum = torch.cumsum(torch.Tensor(tp[cl]), 0)
        fp_cumsum = torch.cumsum(torch.Tensor(fp[cl]), 0)
        gt_cumsum = torch.cumsum(torch.Tensor(gts[cl]), 0)
        pos_det = max(tp_cumsum) + max(fp_cumsum)
        # precision (tp / tp+fp)
        # recall (tp+fp / #gt) => gt = tp + fn
        # avoid div by 0 with .clamp(min=1e-12)
        rec = tp_cumsum / gt_cumsum.clamp(min=1e-12)
        prec = tp_cumsum / (tp_cumsum + fp_cumsum).clamp(min=1e-12)
        ap[cl] = voc_ap(rec, prec)
        recall[cl] = max(rec)
        precision[cl] = max(prec)
        print('class', cl, 'rec', recall[cl],
              'prec', precision[cl], 'AP', ap[cl],
              'tp', sum(tp[cl]), 'fp', sum(fp[cl]), 'gt', sum(gts[cl]))
    # mAP = mean of APs for all classes
    mAP = sum(ap.values()) / len(ap)
    return mAP

In [16]:
trained_model = 'weights/ssd_300_VOC0712.pth'

# load net
net = build_ssd('test', 300, 21)    # initialize SSD
net.load_state_dict(torch.load(trained_model))
net.eval()
print('Finished loading model!')
# load data
valset = VOCDetection(VOCroot, 'val', None, AnnotationTransform())
# evaluation
test_net(net, False, valset, base_transform(net.size,(104,117,123)), 5)

Finished loading model!
Evaluating image 1/2510....
Evaluating image 2/2510....
Evaluating image 3/2510....
Evaluating image 4/2510....
Evaluating image 5/2510....
class 1 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 2 gt 0
class 2 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 2 gt 0
class 3 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 5 gt 0
class 4 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 2 gt 0
class 5 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 4 gt 0
class 6 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 2 gt 0
class 7 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 3 gt 0
class 8 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 9 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 4 gt 0
class 10 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 11 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 12 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 2 gt 0
class 13 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 15 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 4 gt 0
class 16 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 4 gt 0
class 18 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 19 rec 0.0 prec 0.0 AP 0.0 tp 0 fp 1 gt 0
class 20 rec 0.0 prec 0.0 AP 

0.0

In [ ]:
d = defaultdict(list)
d[1].extend([1] * 4)
d

In [ ]:
len(d[1])